## Библиотеки

Нам нужны следующие библиотеки:
- `pandas` - с этой уже знакомы, правда?
- `DecisionTreeClassifier` - для обучения модели (решения)
- `sklearn.model_selection` - для проверки (валидации) модели 
- `gc` прямой вызов "уборщика", чтобы освободить память от "мусора" (RAM)

In [ ]:
import pandas as pd
import numpy as np
 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GroupKFold

import gc

## Загрузим данные

Здесь без изменений.

In [ ]:
df = pd.read_hdf("../input/data.h5")

print(df.shape)
df.sample(5)

(820906, 9)


,order_id,customer_id,product_id,quantity,price_unit,price_total,country_id,order_date,is_canceled
215576,15138,1048,1019,24,39,936,0,2011-08-15 09:57:00,False
134395,9498,66,824,6,1275,7650,6,2011-05-16 13:20:00,False
721986,47261,307,663,3,42,126,0,2010-10-17 16:16:00,False
344684,22105,66,144,12,85,1020,6,2011-11-11 12:52:00,False
537825,34788,775,280,12,165,1980,0,2010-04-26 18:06:00,False


## 🎯 Целевая переменная

Нам нужно определить, что мы хотим прогнозировать. Мы можем прогнозировать разные вещи, а модель машинного обучения с радостью нам в этом "постарается" помочь. Однако мы должны показать нашей модели, чего мы хотим достигнуть! Можешь думать так: *машинное обучение* (ML) - такая золотая рыбка 🐡 (алладин или другой волшебный персонаж), которая пытается выполнить то, что мы хотим, но нам нужно знать, чего мы хотим :). На самом деле - это довольно-таки трудная задача! И правильно сформулировать задачу -  это уже половина или даже более успеха. 


Эйншнтейн как-то сказал:
<blockquote>
Если бы мне дали час на решение задачи, от которой зависела бы моя жизнь, то 55 минут я бы потратил на то, чтобы точно сформулировать вопрос. А для того, чтобы ответить правильно на поставленный вопрос, мне нужно не больше пяти минут.    
</blockquote>


Именно поэтому в предыдущие дни мы "копались" в данных, чтобы найти что-то интересное, с точки зрения эффективности решения и ценностей. Одна из задач может быть сформулирована так:

предположим, для начала мы хотим предсказать, принадлежит ли клиент "X" к сегменту `most_revenue_customer` (т.е. группа около 20% клиентов, которая дает около 80% дохода). 
Наша целевая переменная будет - `is_top_customer` и значения в ней будут: да (`True`) или нет (`False`).
Чтобы просчитать - нам нужно снова создать набор `top_customers` (как мы это делали недавно).

In [ ]:
df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)


df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc


top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())

del df_customers
gc.collect()

0

Основной код с предыдущего дня. Но немного повторим.


```
df_customers = (
    df[ ["price_total", "customer_id"] ]
    .groupby("customer_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "customer_price_total"})
)
```

1. `df[ ["price_total", "customer_id"] ]` - берем из `DataFrame` два интересующих нас столбца `["price_total", "customer_id"]`
2. `.groupby("customer_id")` - группируем по `customer_id` (т.е. собираем клиентов в отдельные "корзинки")
3. `.agg("sum")` - считаем сумму `price_total` для каждого клиента (в итоге получаем сумму, которую потратил каждый клиент в интернет-магазине)
4. `.reset_index()` - из секвенции делаем табличку (чисто технический трюк)
5. `.sort_values(by="price_total", ascending=False)` - сортируем клиентов по убыванию (именно за убывание отвечает параметр `ascending=False`)
7. `.rename(columns={"price_total": "customer_price_total"})` - переименовываем название столбца из `price_total` в `customer_price_total`

Итог всего посчитанного записываем в переменную `df_customers`.


В результате получаем табличку (`DataFrame`), в которой есть два столбика:
- `customer_id` - ID клиента
- `customer_price_total` - общая сумма, которую клиент потратил в магазине

Для каждого клиента - одна строчка.

Идем дальше

```
df_customers["cumsum"] = df_customers["customer_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_customers["most_revenue_customer"] = df_customers["cumsum"] < value_80prc
```

Далее мы хотим найти группы наиболее "щедрых" клиентов, которые с пропорции ~20% принесли около 80% дохода. Поэтому, для начала считаем "кумулятивную сумму". Иными словами - берем первого щедрого клиента и суммируем его со вторым, потом добавляем третьего и т.д. До момента, пока не увидишь, что получившаяся сумма, составляет 80% дохода. 

Напоследок создаем новый столбец `df_customers["most_revenue_customer"]`, где будут два возможных значения: да (`True`) и нет (`False`). Самые "щедрые" клиенты получат значение - `True`.

И теперь можем отфильтровать по ID и собрать в один набор всех `top_customers`. Именно это и делает данная строчка:
`top_customers = set(df_customers[ df_customers["most_revenue_customer"] ]["customer_id"].unique())`

можем разложить ее на меньшие кусочки:
1. `df_customers[ df_customers["most_revenue_customer"] ]` - оставляем только строки с самыми "щедрыми" клиентами
2. `["customer_id"]` - в таблице, которая осталась, "вытягиваем" только столбец `customer_id` - там наши ID клиентов
3. `.unique()` - эта функция возвращает нам только уникальные ID клиентов

И в конце мы еще добавили `set()` - т.е. делаем уникальный набор.

И все это записываем в переменную `top_customers`. В результате у нас есть набор (это уникальный список, т.е. каждый клиент будет только раз).

В самом конце мы "чистили мусор" в памяти:

```
del df_customers
gc.collect()
```

Это для того, чтобы освободить память, например, переменная `df_customers` нам больше не нужна, она была нужна нам для того, чтобы получить `top_customers`. Мы получили. Ключевое слово `del`- освобождает ресурсы (помечает, что уже эта переменная не нужна) и дает право "уборщику" почистить. Процесс запуска уборщика - автоматический, но мы можем его "смотивировать" убрать как можно скорее, вызвав `gc.collect ()`. Обращаю Твое внимание на это, потому что этот практический "трюк", скорее всего, будет частым Твоим другом на практике. Поэтому стоит подружиться заранее.

Так к слову, (для домохозяинов и домохозяек),  ИТ-мир настолько классный, что если хочешь прибраться 🗑️, достаточно вызвать "Мистера Пропера" из мира IT - `gc.collect ()` и все 💪.

Постарайся разобраться в конструкции, которую обсудили, так как далее нужно будет ее немного менять. Если есть вопросы - пиши смело в Slack! Сегодня у нас четвертый день - поэтому вопросы, сомнения и т.д., писать нужно в [#dwthon_day4](https://bit.ly/3fRIQ5X).

## Клиенты
Теперь у нас есть таблица для каждого клиента с его растратами (сумма покупок). Другими словами, каждая строчка - это уникальный клиент. 

Дополнительно сделаем более сложную агрегацию, потому что пришло время делать более сложные вещи :).

Например: `count_orders = (" order_id ", lambda x: len (set (x)))`

Эта запись означает, что появится новый столбец в нашей таблице с названием `count_orders`. Но, чтобы получить этот столбец, мы берем исходный столбец с именем `order_id` и используем агрегирующую функцию `lambda x: len (set (x))`. Конкретно в этом случае - мы подсчитываем количество уникальных заказов для каждого клиента. Например, Саша сделал 3 заказа, а Маша сделала 5 заказов и т.д. Цифры 3 или 5 будут приписаны новому столбцу `count_orders`.

##### ☝️Внимание! Если сложно читать и понимать то, что написано внутри функции `agg`, не беспокойся об этом! 
Ниже будет такой код.
```
.agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("product_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    )
```
На данный момент не так важно понимать все и, тем более, -  как именно это написано на уровне кода. Важно понимать, что мы хотим подсчитывать для каждого клиента, сколько у него заказов, сколько уникальных товаров он купил и т. д. Главное - пойми идею - а сам код сейчас не самое важное, тем более - этому всегда можно доучиться.


### Пишем код!

In [ ]:
df_customers = (
    df
    .groupby("customer_id", as_index=False)
    .agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("product_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    )
)

df_customers["most_revenue_customer"] = df_customers["customer_id"].map(lambda x: x in top_customers)

df_customers

,customer_id,count_orders,count_unq_products,sum_quantity,sum_price_unit,sum_price_total,count_unq_countries,prob_canceled,most_revenue_customer
0,0,159,93,22976,895827,5746671,1,0.010966,True
1,1,40,184,3514,205962,910113,1,0.097727,True
2,2,30,139,8971,95500,1337464,1,0.009050,True
3,3,9,40,1108,18626,226381,1,0.000000,False
4,4,14,11,390,20028,346978,1,0.307692,True
...,...,...,...,...,...,...,...,...,...
5874,5874,2,46,213,15654,48343,1,0.040000,False
5875,5875,1,21,261,4944,30081,1,0.000000,False
5876,5876,1,69,367,33622,110868,1,0.000000,False
5877,5877,1,33,85,7995,13458,1,0.000000,False


Давай вместе посмотрим - что внутри `df_customers` таблицы (`DataFrame`).

- `customer_id` - ID клиента.
- `count_orders` - количество уникальных заказов за все время для данного клиента.
    - `count_unq_products` - количество уникальных продуктов (например, Саша покупает только хлеб и молоко, значит - 2 уникальных продукта) за все время в разрезе каждого клиента.
- `sum_quantity` - сумму покупаемых товаров в штуках (например, Саша купил 2 шт. хлеба и 3 шт. молока - сумма будет 5.
- `sum_price_unit` - сумма для каждого вида продукта (например, у Саши 2 вида продукта - поэтому это будет сумма молока и хлеба - игнорируются штуки закупленных продуктов.
- `sum_price_total` - общая сумма затрат за все время истории в данных - для каждого клиента отдельно.
- `count_unq_countries` - из каких стран покупал клиент (обычно одна страна, но иногда бывает больше).
- `prob_canceled` - вероятность того, что заказ будет возвращен (здесь простая формула - кол-во заказов, которые клиент вернул, делим на количество заказов, которые он купил). Значение может быть между 0 и 1. Где 0 - это значит, что данный клиент никогда не возвращал заказы, а 1 -  клиент вернул все свои заказы.
- `most_revenue_customer` - информация о том, принадлежит ли клиент к группе самых щедрых клиентов.

Для создания нашей целевой переменной будем использовать столбец `most_revenue_customer`. Давай перепроверим - какой процент клиентов принадлежит группе `most_revenue_customer`.

In [ ]:
df_customers["most_revenue_customer"].value_counts(normalize=True)

False    0.774792
True     0.225208
Name: most_revenue_customer, dtype: float64

У нас 22,5% клиентов принадлежат к категории `top_customers`.

Хорошо! Пора тренировать модель.

# Краткий глоссарий ML


## Что такое алгоритм (например, деревья решений)?
Это определенная последовательность шагов, которые будут выполнены для обучения модели (и лучшего ее изучения).


## Что такое модель?
Это определенное «состояние» того, что было обнаружено в исторических данных.

Пример по аналогии.

Алгоритм - это рецепт блюда (даже больше набор ингридиентов). 
Модель - реализованное состояние этого рецепта, т.е. готовое блюдо. 


В этой модели подготовлены данные.
Рецепт всего один, а блюд могут быть миллионы и они будут разными, в зависимости от того, что было подано «на входе» (брошено в чашу).


## Что значит обучить модель?
Это процесс, когда модель ищет зависимости (корреляции) в исторических данных. Результат обучения - это готовая модель, которую мы затем можем использовать для предсказания будущего (или иначе - существуют разные творческие подходы, как можно ее использовать).

## Что такое признаки?
Это свойства (характеристики) объектов. В нашем случае характеристиками для покупателя могут быть: 
- количество заказов, 
- количество  купленных товаров
- и т.д.

Можно сказать, что признаки - это столбцы в нашей таблице (конечно, без целевой переменной, в нашем случае, без `most_revenue_customer`).

## Что такое целевая переменная?
Это то, что предсказывает модель.
Можно сказать, что это тот столбец в нашей таблице, который нас интересует. В нашем случае - `most_revenue_customer`, т.е. нас интересует описание клиента (его признаки), которые помогут определить -  будет ли клиент в группе наиболее "щедрых" клиентов или нет.


## Почему целевая переменная не может быть в признаках?
Современный алгоритм «умный». Он сразу «поймает», что это ответ, и будет использовать этот признак, как ответ. Но какова в этом ценность? Если у нее в признаках (описаниях) заранее "зашит" ответ - толку на основании этого прогнозировать ответ. Нет смысла прогонять это через модель. 

Пример.
*Это все равно, что студент на экзамене вместе с вытянутым вопросом получает ответ. Студент конечно же может не подсматривать в 100% правильный ответ и думать сам, но все же прагматичнее переписать и минимизировать риски (а машинное обучение - это прежде всего прагматичная оптимизация).*


## Зачем мы тренируем модель?
Чтобы выловить неочевидные взаимосвязи и затем - делать прогнозы. Модель тренируется только на основании признаков, а ответ записывается в целевую переменную. Хотя стоит заметить, что может быть по-разному. В нашем случае речь идет о более творческом подходе, и это именно то, что будет завтра -  и что стоит обязательно выполнить. Сегодня давай сосредоточимся на обучении модели по метрике успеха.


## Что такое метрика успеха?
Это способ измерить качество работы нашего решения (модели). 

Простой пример.
У нас есть две (разные) модели. Каждая из моделей дает свой результат - в итоге у нас есть два разных результата. Вопрос: как мы можем сказать, у какой модели лучший результат? Метрика успеха сводится к «сжатию» качества модели в конкретное число. Иными словами, очень легко сравнивать конкретные числа, например, этими числами является точность.

Точность - это один из примеров метрики. Например, модель A дает точность 95%, модель Б - 99%. В такоме случае (для этой метрики), модель Б лучше. Хотя в метрике есть множество нюансов, которые нужно понимать, не будем сейчас вдаваться в подробности.


Ну что, теперь мы подготовим признаки и будем тренировать модель.

Ниже приведен код, который делает простую вещь, а именно - оставляет только столбцы, которые являются числовыми или  логическими (анг. `boolean`) - то есть столбцы, которые принимают значения: `True` или `False`.

In [ ]:
feats = list(df_customers.select_dtypes([np.number, bool]).columns)

feats

['customer_id',
 'count_orders',
 'count_unq_products',
 'sum_quantity',
 'sum_price_unit',
 'sum_price_total',
 'count_unq_countries',
 'prob_canceled',
 'most_revenue_customer']

Как видно, среди признаков также "выловили" `most_revenue_customer`, который (напомню) является нашей целевой переменной. 
Его следует удалить. Это уже объяснял почему, если забыл(а)- почитай выше (не хотим давать однозначный ответ для студнета на экзамене - это как-то не логично). 

Проблему можно легко решить, cоздав список исключений. На данный момент в этом списке будет только один столбец `most_revenue_customer`, но можно потом добавить больше.

In [ ]:
black_list = ["most_revenue_customer"]
feats = [x for x in feats if x not in black_list]

У нас остался минимальный шаг, чтобы получить так называемые `X` и `y`.

- `X` - это наша таблица (обычно ее называют в этом случае как - матрица) признаков.
- `y` - это наша целевая переменная

In [ ]:
X = df_customers[feats].values
y = df_customers["most_revenue_customer"].values

### Обучаем модель

Сначала мы воспользуемся деревом решений. Как я уже упоминал ранее, не нужно заранее знать, как работает этот алгоритм. Для начала можешь думать об этом, как о решении типа «черный ящик», которое может учиться «как-то», на основании данных. Главное, что дает результат.

In [ ]:
scores = cross_val_score(DecisionTreeClassifier(max_depth=3), X, y, scoring="accuracy", cv=5)
np.mean(scores), np.std(scores)

(0.9996598639455783, 0.0004165798882284457)

Получили очень хороший результат: точность 99%. Поздравляю! Твоя первая модель только что создана. Хотя качество достаточно подозрительно хорошее, скорее всего - у нас имеется какая-та сильная корреляция (взаимосвязь). 


Сейчас об этом не стоит сильно беспокоиться. Мы еще к этому вернемся "завтра", где займемся интерпретацией модели.

## Задание 4.1

Мы только что обучили модель для `top_customers`. Твоя задача - обучить новую модель, используя `top_products`


### 💡 Советы: 

1. Сначала подготовь набор `top_products`.
2. Затем подготовь `df_products`, который будет включать полезные признаки (конечно, можешь вдохновиться тем, как мы делали для` df_customers`).
3. Затем приготовь `feats`, `X` i `y`.
4. Обучи модель (например, `DecisionTreeClassifier`).
5. Поделись своим результатом :)

In [ ]:
df_products = (
    df[ ["price_total", "product_id"] ]
    .groupby("product_id")
    .agg("sum")
    .reset_index()
    .sort_values(by="price_total", ascending=False)
    .rename(columns={"price_total": "product_price_total"})
)


df_products["cumsum"] = df_products["product_price_total"].cumsum()
value_80prc = int(df["price_total"].sum() * 0.8)
df_products["most_revenue_products"] = df_products["cumsum"] < value_80prc


top_products = set(df_products[ df_products["most_revenue_products"] ]["product_id"].unique())

del df_products
gc.collect()

df_products = (
    df
    .groupby("product_id", as_index=False)
    .agg(
        count_orders=("order_id", lambda x: len(set(x))),
        count_unq_products=("product_id", lambda x: len(set(x))),
        sum_quantity=("quantity", np.sum),
        sum_price_unit=("price_unit", np.sum),
        sum_price_total=("price_total", np.sum),
        count_unq_countries=("country_id", lambda x: len(set(x))),
        prob_canceled=("is_canceled", np.mean)
    )
)

df_products["most_revenue_products"] = df_products["product_id"].map(lambda x: x in top_products)

df_products["most_revenue_products"].value_counts(normalize=True)

feats = list(df_products.select_dtypes([np.number, bool]).columns)

black_list = ["most_revenue_products"]
feats = [x for x in feats if x not in black_list]

X = df_products[feats].values
y = df_products["most_revenue_products"].values

scores = cross_val_score(DecisionTreeClassifier(max_depth=3), X, y, scoring="accuracy", cv=5)
np.mean(scores), np.std(scores)

(0.999484203525108, 0.0006317193061419072)

## 🧠 Включим критическое мышление


Давай подумаем вместе, что именно мы сейчас делаем (вот именно, что это было?).

У нас есть список наших клиентов на момент X (последняя покупка). Посмотримм на ось времени. Лучше сразу разберем это на примере.

In [ ]:
df["order_date"].min(), df["order_date"].max()

(Timestamp('2009-12-01 07:45:00'), Timestamp('2011-12-09 12:50:00'))

У нас есть данные с `2009-12-01 07:45:00` по `2011-12-09 12:50:00`.

Когда мы обучаем модель, мы передаем модели всю историю, которая у нас есть о клиенте. Все это означает, что модель учится на ситуациях, когда мы уже «довольно много» знаем о клиенте. Фактически, может оказаться, что - зачем в таком случае ML? Зачем использовать трудные алгоритмы, если можно на калькуляторе посчитать сколько потратил клиент и по факту приписать его к `top_customer`.

Было бы конечно интереснее обучить модель немного по-другому и выявить ее потенциал в самом начале.

Давай подумаем об этом так.

У нас есть история о клиенте, например, за 2 года. Мы знаем, что он находится в сегменте `top_customer`. Однако этот клиент присоединился к этому сегменту... скажем, через 15 месяцев. Значит, на временной оси между 0 и 15 месяцев не было понятно - будет ли данный клиент в `top_customer`, но уже, набрав критическую массу (клиент потратил X денег), клиент присоединяется к клубу `top_customer`, из которого не исключают. Получается, что предсказывать судьбу этого клиента на оси времени через 15 месяцев,  становится мало интересной задачей - т.к. ответ остается без изменений.


### Что мы можем сделать?
Что ж - это хороший вопрос. Понимание оси времени в машинном обучении очень важно, и, по моему опыту, именно здесь совершается больше всего ошибок, особенно новичками. Поэтому заостряю на конкретном примере Твое внимание. Надо подумать что дальше :).


## 🧠 Включим критическое мышление
Давай думать, как мы можем воспользоваться нашей ситуацией.

1. Мы знаем, чем это «заканчивается» (если клиент уже в `top_customers`, то там уже остается), но мы еще не знаем, кто будет там в будущем (у нас пока нет этих данных).
2. Но мы хотим показать модели только часть данных (например, мы знаем историю двухлетней давности, но мы показываем модели только первые 10 месяцев или 12 месяцев, или 15 месяцев, или другой разумный для нас промежуток времени).

Подумай, как мы можем обучить модель, чтобы, с одной стороны, она получала как можно меньше информации (то есть могла «распознать» потенциал, как можно скорее). Какие плюсы и минусы (по крайней мере, на Твой взгляд).


#### ☝️Обрати внимание, что речь идет не о технических «размышлениях» - как это сделать (Ты можешь всегда попросить помощи в чатике). Подумай об этом концептуально, т.е. чего мы хотим достичь :)

Что нас волнует? Предполагается ли, что модель учится на уже более продвинутом «состоянии» клиента, когда история достаточно длинная (например, 1-2 года, и поэтому уже известно, кто есть кто), или модель умеет находить «золотой потенциал» намного раньше. Последнее звучит как нечто с гораздо большей добавленной стоимостью. И здесь машинное обучение может иметь преимущество перед «классической аналитикой».